In [ ]:
import cv2
import numpy as np
import os
import subprocess
import time
from threading import Thread

# Paths
WAV2LIP_PATH = "Wav2Lip"
INPUT_IMAGE = "inputs/sample.webp"
OUTPUT_VIDEO = "outputs/output.mp4"
AUDIO_FILE = "inputs/audio.wav"

# Function to run Wav2Lip inference
def run_wav2lip(audio_path, img_path, output_path):
    command = f"python {WAV2LIP_PATH}/inference.py --checkpoint_path {WAV2LIP_PATH}/Wav2Lip.pth --face {img_path} --audio {audio_path} --outfile {output_path}"
    subprocess.call(command, shell=True)

# Function to capture audio and save as audio.wav
def record_audio(audio_path, duration=5):
    import sounddevice as sd
    from scipy.io.wavfile import write
    
    fs = 44100  # Sampling frequency
    print("Recording...")
    audio_data = sd.rec(int(duration * fs), samplerate=fs, channels=2, dtype='int16')
    sd.wait()
    write(audio_path, fs, audio_data)
    print(f"Audio saved at {audio_path}")

# Main loop to capture real-time video and convert to avatar
def real_time_avatar():
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Camera not accessible!")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture video.")
            break

        # Show real-time camera feed
        cv2.imshow('Real-Time Avatar', frame)

        # Press 'r' to record and generate avatar lip-sync
        if cv2.waitKey(1) & 0xFF == ord('r'):
            record_audio(AUDIO_FILE)
            
            # Run Wav2Lip
            run_wav2lip(AUDIO_FILE, INPUT_IMAGE, OUTPUT_VIDEO)
            
            # Play the generated avatar video
            play_video(OUTPUT_VIDEO)
        
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Function to play the generated avatar video
def play_video(video_path):
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        cv2.imshow('Avatar Output', frame)
        
        # Press 'q' to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    real_time_avatar()


In [ ]:
import os

# Define input file paths
image_path = "Wav2Lip/inputs/sample.webp"  # Path to your image file
audio_path = "Wav2Lip/inputs/harvard.wav"   # Path to your audio file
output_path = "output.mp4"  # Path for the generated video

# Command to run Wav2Lip
command = f"python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face {image_path} --audio {audio_path} --outfile {output_path}"

# Run the command
os.system(command)

# Check if the output file is generated
if os.path.exists(output_path):
    print(f"Video generated successfully: {output_path}")
else:
    print("Failed to generate video. Check your input files or model path.")


In [ ]:
import openai

# Set your API key
openai.api_key = "your-api-key"

# Define the model and prompt
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",  # or "gpt-4" if using GPT-4
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello! How are you?"}
  ]
)

# Print the response
print(response['choices'][0]['message']['content'])
